In [1]:
"""

Path tracking simulation with iterative linear model predictive control for speed and steer control

author: Atsushi Sakai (@Atsushi_twi)

"""
%matplotlib tk
import matplotlib.pyplot as plt
import cvxpy
import math
import numpy as np
import cubic_spline_planner


NX = 4  # x = x, y, v, yaw
NU = 2  # a = [accel, steer]
T = 5  # horizon length

# mpc parameters
R = np.diag([0.01, 0.01])  # input cost matrix
Rd = np.diag([0.01, 1.0])  # input difference cost matrix
Q = np.diag([1.0, 1.0, 0.5, 0.5])  # state cost matrix
Qf = Q  # state final matrix
GOAL_DIS = 1.5  # goal distance
STOP_SPEED = 0.5 / 3.6  # stop speed
MAX_TIME = 500.0  # max simulation time

# iterative paramter
MAX_ITER = 3  # Max iteration
DU_TH = 0.1  # iteration finish param

TARGET_SPEED = 10.0 / 3.6  # [m/s] target speed
N_IND_SEARCH = 10  # Search index number

DT = 0.2  # [s] time tick

# Vehicle parameters
LENGTH = 4.5  # [m]
WIDTH = 2.0  # [m]
BACKTOWHEEL = 1.0  # [m]
WHEEL_LEN = 0.3  # [m]
WHEEL_WIDTH = 0.2  # [m]
TREAD = 0.7  # [m]
WB = 2.5  # [m]

MAX_STEER = np.deg2rad(45.0)  # maximum steering angle [rad]
MAX_DSTEER = np.deg2rad(30.0)  # maximum steering speed [rad/s]
MAX_SPEED = 55.0 / 3.6  # maximum speed [m/s]
MIN_SPEED = -20.0 / 3.6  # minimum speed [m/s]
MAX_ACCEL = 1.0  # maximum accel [m/ss]

show_animation = True



        

def pi_2_pi(angle):
    while(angle > math.pi):
        angle = angle - 2.0 * math.pi

    while(angle < -math.pi):
        angle = angle + 2.0 * math.pi

    return angle





def plot_car(x, y, yaw, steer=0.0, cabcolor="-r", truckcolor="-k"):  # pragma: no cover

    outline = np.array([[-BACKTOWHEEL, (LENGTH - BACKTOWHEEL), (LENGTH - BACKTOWHEEL), -BACKTOWHEEL, -BACKTOWHEEL],
                        [WIDTH / 2, WIDTH / 2, - WIDTH / 2, -WIDTH / 2, WIDTH / 2]])

    fr_wheel = np.array([[WHEEL_LEN, -WHEEL_LEN, -WHEEL_LEN, WHEEL_LEN, WHEEL_LEN],
                         [-WHEEL_WIDTH - TREAD, -WHEEL_WIDTH - TREAD, WHEEL_WIDTH - TREAD, WHEEL_WIDTH - TREAD, -WHEEL_WIDTH - TREAD]])

    rr_wheel = np.copy(fr_wheel)

    fl_wheel = np.copy(fr_wheel)
    fl_wheel[1, :] *= -1
    rl_wheel = np.copy(rr_wheel)
    rl_wheel[1, :] *= -1

    Rot1 = np.array([[math.cos(yaw), math.sin(yaw)],
                     [-math.sin(yaw), math.cos(yaw)]])
    Rot2 = np.array([[math.cos(steer), math.sin(steer)],
                     [-math.sin(steer), math.cos(steer)]])

    fr_wheel = (fr_wheel.T.dot(Rot2)).T
    fl_wheel = (fl_wheel.T.dot(Rot2)).T
    fr_wheel[0, :] += WB
    fl_wheel[0, :] += WB

    fr_wheel = (fr_wheel.T.dot(Rot1)).T
    fl_wheel = (fl_wheel.T.dot(Rot1)).T

    outline = (outline.T.dot(Rot1)).T
    rr_wheel = (rr_wheel.T.dot(Rot1)).T
    rl_wheel = (rl_wheel.T.dot(Rot1)).T

    outline[0, :] += x
    outline[1, :] += y
    fr_wheel[0, :] += x
    fr_wheel[1, :] += y
    rr_wheel[0, :] += x
    rr_wheel[1, :] += y
    fl_wheel[0, :] += x
    fl_wheel[1, :] += y
    rl_wheel[0, :] += x
    rl_wheel[1, :] += y

    plt.plot(np.array(outline[0, :]).flatten(),
             np.array(outline[1, :]).flatten(), truckcolor)
    plt.plot(np.array(fr_wheel[0, :]).flatten(),
             np.array(fr_wheel[1, :]).flatten(), truckcolor)
    plt.plot(np.array(rr_wheel[0, :]).flatten(),
             np.array(rr_wheel[1, :]).flatten(), truckcolor)
    plt.plot(np.array(fl_wheel[0, :]).flatten(),
             np.array(fl_wheel[1, :]).flatten(), truckcolor)
    plt.plot(np.array(rl_wheel[0, :]).flatten(),
             np.array(rl_wheel[1, :]).flatten(), truckcolor)
    plt.plot(x, y, "*")




def get_nparray_from_matrix(x):
    return np.array(x).flatten()


def get_closest_point(state, ccx, ccy, cyaw, pind):
    
    d = [(state.x-idx) ** 2 + (state.y-idy) ** 2 for (idx, idy) in zip(ccx,ccy)]
    
    mind = min(d)
    ind = d.index(mind) + pind
    print('ds',ind,mind)
    return ind, mind




def ccalc_ref_trajectory(state, cx, cy, cyaw, ck, sp, dl, pind):
    xref = np.zeros((NX, T + 1))
    dref = np.zeros((1, T + 1))
    ncourse = len(cx)

    ind, _ = get_closest_point(state, cx, cy, cyaw, pind)

    if pind >= ind:
        ind = pind

    xref[0, 0] = cx[ind]
    xref[1, 0] = cy[ind]
    xref[2, 0] = sp[ind]
    xref[3, 0] = cyaw[ind]
    dref[0, 0] = 0.0  # steer operational point should be 0

    S = 0.0
    vel=state.v
    for i in range(T + 1):
        S += abs(vel) * DT
        vel+=acc*DT
        dind = int(round(travel / dl))

        if (ind + dind) < ncourse:
            xref[0, i] = cx[ind + dind]
            xref[1, i] = cy[ind + dind]
            xref[2, i] = sp[ind + dind]
            xref[3, i] = cyaw[ind + dind]
            dref[0, i] = 0.0
        else:
            xref[0, i] = cx[ncourse - 1]
            xref[1, i] = cy[ncourse - 1]
            xref[2, i] = sp[ncourse - 1]
            xref[3, i] = cyaw[ncourse - 1]
            dref[0, i] = 0.0

    return xref, ind, dref

def calc_ref_trajectory(state, ccx, ccy, ccyaw, ck, ssp, dl, ind):
    xref = np.zeros((NX, T + 1))
    dref = np.zeros((1, T + 1))
    ncourse = len(cx)

    #ind, _ = get_closest_point(state, ccx, ccy, ccyaw, pind)
    #if pind >= ind:
    #    ind = pind

    xref[0, 0] = ccx[0]
    xref[1, 0] = ccy[0]
    xref[2, 0] = ssp[0]
    xref[3, 0] = ccyaw[0]
    dref[0, 0] = 0.0  # steer operational point should be 0

    travel = 0.0

    for i in range(T + 1):
        travel += abs(state.v) * DT
        dind = int(round(travel / dl))

        if (dind) < len(ccx)-1:
            xref[0, i] = ccx[dind]
            xref[1, i] = ccy[dind]
            xref[2, i] = ssp[dind]
            xref[3, i] = ccyaw[dind]
            dref[0, i] = 0.0
        else:
            xref[0, i] = ccx[-1]
            xref[1, i] = ccy[-1]
            xref[2, i] = ssp[- 1]
            xref[3, i] = ccyaw[- 1]
            dref[0, i] = 0.0

    return xref, ind, dref

def check_goal(state, goal, tind, nind):

    # check goal
    dx = state.x - goal[0]
    dy = state.y - goal[1]
    d = math.sqrt(dx ** 2 + dy ** 2)

    isgoal = (d <= GOAL_DIS)

    if abs(tind - nind) >= 5:
        isgoal = False

    isstop = (abs(state.v) <= STOP_SPEED)

    if isgoal and isstop:
        return True

    return False




def calc_speed_profile(cx, cy, cyaw, target_speed):

    speed_profile = [target_speed] * len(cx)
    direction = 1.0  # forward

    # Set stop point
    for i in range(len(cx) - 1):
        dx = cx[i + 1] - cx[i]
        dy = cy[i + 1] - cy[i]

        move_direction = math.atan2(dy, dx)

        if dx != 0.0 and dy != 0.0:
            dangle = abs(pi_2_pi(move_direction - cyaw[i]))
            if dangle >= math.pi / 4.0:
                direction = -1.0
            else:
                direction = 1.0

        if direction != 1.0:
            speed_profile[i] = - target_speed
        else:
            speed_profile[i] = target_speed

    speed_profile[-1] = 0.0

    return speed_profile


def smooth_yaw(yaw):

    for i in range(len(yaw) - 1):
        dyaw = yaw[i + 1] - yaw[i]

        while dyaw >= math.pi / 2.0:
            yaw[i + 1] -= math.pi * 2.0
            dyaw = yaw[i + 1] - yaw[i]

        while dyaw <= -math.pi / 2.0:
            yaw[i + 1] += math.pi * 2.0
            dyaw = yaw[i + 1] - yaw[i]

    return yaw


def get_straight_course(dl):
    ax = [0.0, 5.0, 10.0, 20.0, 30.0, 40.0, 50.0]
    ay = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
    cx, cy, cyaw, ck, s = cubic_spline_planner.calc_spline_course(
        ax, ay, ds=dl)

    return cx, cy, cyaw, ck


def get_straight_course2(dl):
    ax = [0.0, -10.0, -20.0, -40.0, -50.0, -60.0, -70.0]
    ay = [0.0, -1.0, 1.0, 0.0, -1.0, 1.0, 0.0]
    cx, cy, cyaw, ck, s = cubic_spline_planner.calc_spline_course(
        ax, ay, ds=dl)

    return cx, cy, cyaw, ck


def get_straight_course3(dl):
    ax = [0.0, -10.0, -20.0, -40.0, -50.0, -60.0, -70.0]
    ay = [0.0, -1.0, 1.0, 0.0, -1.0, 1.0, 0.0]
    cx, cy, cyaw, ck, s = cubic_spline_planner.calc_spline_course(
        ax, ay, ds=dl)

    cyaw = [i - math.pi for i in cyaw]

    return cx, cy, cyaw, ck


def get_forward_course(dl):
    ax = [0.0, 60.0, 125.0, 50.0, 75.0, 30.0, -10.0]
    ay = [0.0, 0.0, 50.0, 65.0, 30.0, 50.0, -20.0]
    cx, cy, cyaw, ck, s = cubic_spline_planner.calc_spline_course(
        ax, ay, ds=dl)

    return cx, cy, cyaw, ck


def get_switch_back_course(dl):
    ax = [0.0, 30.0, 6.0, 20.0, 35.0]
    ay = [0.0, 0.0, 20.0, 35.0, 20.0]
    cx, cy, cyaw, ck, s = cubic_spline_planner.calc_spline_course(
        ax, ay, ds=dl)
    ax = [35.0, 10.0, 0.0, 0.0]
    ay = [20.0, 30.0, 5.0, 0.0]
    cx2, cy2, cyaw2, ck2, s2 = cubic_spline_planner.calc_spline_course(
        ax, ay, ds=dl)
    cyaw2 = [i - math.pi for i in cyaw2]
    cx.extend(cx2)
    cy.extend(cy2)
    cyaw.extend(cyaw2)
    ck.extend(ck2)

    return cx, cy, cyaw, ck


def main():
    

    t, x, y, yaw, v, d, a = do_simulation(
        cx, cy, cyaw, ck, sp, dl, initial_state)




def main2():
    print(__file__ + " start!!")

    dl = 1.0  # course tick
    cx, cy, cyaw, ck = get_straight_course3(dl)

    sp = calc_speed_profile(cx, cy, cyaw, TARGET_SPEED)

    initial_state = State(x=cx[0], y=cy[0], yaw=0.0, v=0.0)

    t, x, y, yaw, v, d, a = do_simulation(
        cx, cy, cyaw, ck, sp, dl, initial_state)

    if show_animation:  # pragma: no cover
        plt.close("all")
        plt.subplots()
        plt.plot(cx, cy, "-r", label="spline")
        plt.plot(x, y, "-g", label="tracking")
        plt.grid(True)
        plt.axis("equal")
        plt.xlabel("x[m]")
        plt.ylabel("y[m]")
        plt.legend()

        plt.subplots()
        plt.plot(t, v, "-r", label="speed")
        plt.grid(True)
        plt.xlabel("Time [s]")
        plt.ylabel("Speed [kmh]")

        plt.show()

In [2]:
def iterative_linear_mpc_control(xref, Motion_Status, dref, oa, od,x0):
    """
    MPC contorl with updating operational point iteraitvely
    """  
    
    if oa is None or od is None:
        oa = [0.0] * T
        od = [0.0] * T
    state=Motion_Status.get_state()   
    for i in range(MAX_ITER):
        xbar = Motion_Status.get_trajectory(oa, od,T)
        poa, pod = oa[:], od[:]
        oa, od, ox, oy, oyaw, ov = linear_mpc_control(xref, xbar, state, dref,x0)
        du = sum(abs(oa - poa)) + sum(abs(od - pod))  # calc u change value
        if du <= DU_TH:
            break
    else:
        print("Iterative is max iter")

    return oa, od, ox, oy, oyaw, ov

def get_linear_model_matrix(v, phi, delta):

    A = np.zeros((NX, NX))
    A[0, 0] = 1.0
    A[1, 1] = 1.0
    A[2, 2] = 1.0
    A[3, 3] = 1.0
    A[0, 2] = DT * math.cos(phi)
    A[0, 3] = - DT * v * math.sin(phi)#
    A[1, 2] = DT * math.sin(phi)
    A[1, 3] = DT * v * math.cos(phi)#
    A[3, 2] = DT * math.tan(delta) / WB

    B = np.zeros((NX, NU))
    B[2, 0] = DT
    B[3, 1] = DT * v / (WB * math.cos(delta) ** 2)#

    C = np.zeros(NX)
    C[0] = DT * v * math.sin(phi) * phi
    C[1] = - DT * v * math.cos(phi) * phi
    C[3] = - DT * v * delta / (WB * math.cos(delta) ** 2)
    #C = np.zeros(NX)
    return A, B, C

def linear_mpc_control(xref, xbar, state, dref,x0):
    """
    linear mpc control

    xref: reference point
    xbar: operational point
    x0: initial state
    dref: reference steer angle
    """

    x = cvxpy.Variable((NX, T + 1))
    u = cvxpy.Variable((NU, T))

    cost = 0.0
    constraints = []

    for t in range(T):
        cost += cvxpy.quad_form(u[:, t], R)

        if t != 0:
            cost += cvxpy.quad_form(xref[:, t] - x[:, t], Q)

        A, B, C = get_linear_model_matrix(xbar[2, t], xbar[3, t], 0)
        constraints += [x[:, t + 1] == A * x[:, t] + B * u[:, t]+ C]

        if t < (T - 1):
            cost += cvxpy.quad_form(u[:, t + 1] - u[:, t], Rd)
            constraints += [cvxpy.abs(u[1, t + 1] - u[1, t]) <=
                            MAX_DSTEER * DT]

    cost += cvxpy.quad_form(xref[:, T] - x[:, T], Qf)

    constraints += [x[:, 0] == [state.x,state.y,state.v,state.yaw]]
    constraints += [x[2, :] <= MAX_SPEED]
    constraints += [x[2, :] >= MIN_SPEED]
    constraints += [cvxpy.abs(u[0, :]) <= MAX_ACCEL]
    constraints += [cvxpy.abs(u[1, :]) <= MAX_STEER]

    prob = cvxpy.Problem(cvxpy.Minimize(cost), constraints)
    prob.solve(solver=cvxpy.ECOS, verbose=False)

    if prob.status == cvxpy.OPTIMAL or prob.status == cvxpy.OPTIMAL_INACCURATE:
        ox = get_nparray_from_matrix(x.value[0, :])
        oy = get_nparray_from_matrix(x.value[1, :])
        ov = get_nparray_from_matrix(x.value[2, :])
        oyaw = get_nparray_from_matrix(x.value[3, :])
        oa = get_nparray_from_matrix(u.value[0, :])
        odelta = get_nparray_from_matrix(u.value[1, :])

    else:
        print("Error: Cannot solve mpc..")
        oa, odelta, ox, oy, oyaw, ov = None, None, None, None, None, None

    return oa, odelta, ox, oy, oyaw, ov



In [3]:
class State:
     
    def __init__(self, x=0.0, y=0.0, yaw=0.0, v=0.0):
        self.x = x
        self.y = y
        self.yaw = yaw
        self.v = v
        self.predelta = None

class Motion():

    def __init__(self, x=0.0, y=0.0, yaw=0.0, v=0.0):
        self.state=State(x=x, y=y, yaw=yaw, v=v)
        
    def get_state(self):
        return State(x=self.state.x, y=self.state.y, yaw=self.state.yaw, v=self.state.v)
        
    def set_state(self,state):
        self.state=state
    def get_next_state(self, a, delta):

        # input check
        if delta >= MAX_STEER:
            delta = MAX_STEER
        elif delta <= -MAX_STEER:
            delta = -MAX_STEER

        next_x = self.state.x + self.state.v * math.cos(self.state.yaw) * DT
        next_y = self.state.y + self.state.v * math.sin(self.state.yaw) * DT
        next_yaw = self.state.yaw + self.state.v / WB * math.tan(delta) * DT
        next_v = self.state.v + a * DT
        
        if next_v > MAX_SPEED:
            next_v = MAX_SPEED
        elif next_v < MIN_SPEED:
            next_v = MIN_SPEED

        return State(x=next_x,y=next_y,v=next_v,yaw=next_yaw)


    def get_trajectory(self,accels, deltas,time_step):
        xbar =np.zeros((NX, time_step + 1))
        xbar[:, 0] = self.state.x,self.state.y,self.state.v,self.state.yaw
        #_state=State(x=state.x,y=state.y,v=state.v,yaw=state.yaw)
        _state=Motion.get_state(self)
        for (accel, delta, i) in zip(accels, deltas, range(1, T + 1)):
            next_state = Motion.get_next_state(self,accel, delta)
            xbar[:, i] = next_state.x,next_state.y,next_state.v,next_state.yaw

        return xbar


In [4]:
def gen_points(cx,cy):
    dx=2
    dy=1
    pts=[[cx-dx,cy+dy],
    [cx+dx,cy+dy],
    [cx+dx,cy-dy],
    [cx-dx,cy-dy],
    [cx-dx,cy+dy]]
    pts=np.array(pts,dtype=np.float64)
    return pts
class move_points():
    def __init__(self,points,XX,YY,phi):
        self.points=points.copy()
        self.XX=XX
        self.YY=YY
        self.ox=0
        self.oy=0
        phi=(math.pi*phi)/180
        self.rot_mat=np.array([[math.cos(phi),-math.sin(phi)],[math.sin(phi),math.cos(phi)]])
    def move(self):
        self.points[:,0]+=self.XX
        self.points[:,1]+=self.YY
    def get_origin(self):
        ox=0
        oy=0
        oo=np.mean(self.points,axis=0)
        self.ox=oo[0]
        self.oy=oo[1]
        print(self.ox,self.oy)
        return self.ox,self.oy
    def shift_origin(self):
        self.points[:,0]-=np.float64(self.ox)
        self.points[:,1]-=np.float64(self.oy)
        return self.points
    def reverse_shift_origin(self):
        self.points[:,0]+=self.ox
        self.points[:,1]+=self.oy
        return self.points
    def rotate(self):
        for i in range(len(self.points)):
            self.points[i]=self.points[i].dot(self.rot_mat)
        return self.points
    def apply(self):
        self.move()
        self.get_origin()
        self.shift_origin()
        self.rotate()
        self.reverse_shift_origin()
        return self.points

In [5]:
dl = 1.0  # course tick
# cx, cy, cyaw, ck = get_straight_course(dl)
#cx, cy, cyaw, ck = get_straight_course2(dl)
#cx, cy, cyaw, ck = get_straight_course3(dl)
# cx, cy, cyaw, ck = get_forward_course(dl)
cx, cy, cyaw, ck = get_switch_back_course(dl)

sp = calc_speed_profile(cx, cy, cyaw, TARGET_SPEED)

Motion_Status = Motion(x=cx[0], y=cy[0], yaw=cyaw[0], v=0.0)
state=Motion_Status.get_state()
# initial yaw compensation
if state.yaw - cyaw[0] >= math.pi:
        state.yaw -= math.pi * 2.0
elif state.yaw - cyaw[0] <= -math.pi:
        state.yaw += math.pi * 2.0
Motion_Status.set_state(state)
goal = [cx[-1], cy[-1]]


time = 0.0
x = [state.x]
y = [state.y]
yaw = [state.yaw]
v = [state.v]
t = [0.0]
d = [0.0]
a = [0.0]
def get_raw_sensor_value():
    pass
_cx=cx[:10]
_cy=cy[:10]
_cyaw=cyaw[:10]
_sp=sp[:10]
#index, _ = get_closest_point(state, _cx, _cy, cyaw, 0)
odelta, oa = None, None
index=0
cyaw = smooth_yaw(cyaw)
while MAX_TIME >= time:
        state=Motion_Status.get_state()
        index, _ = get_closest_point(state, _cx, _cy, cyaw, index)
        #if pind >= index:
        #    index = pind
        _cx=cx[index:index+15]
        _cy=cy[index:index+15]
        _cyaw=cyaw[index:index+15]
        _sp=sp[index:index+15]
        
        pind=index
        xref,_, dref = calc_ref_trajectory(state, _cx, _cy,_cyaw, ck, _sp, dl, index)
        print('index',pind,index)
        x0 = [state.x, state.y, state.v, state.yaw]  # current state

        oa, odelta, ox, oy, oyaw, ov = iterative_linear_mpc_control(
            xref, Motion_Status, dref, oa, odelta,x0)

        if odelta is not None:
            di, ai = odelta[0], oa[0]

        next_state = Motion_Status.get_next_state(ai, di)
        Motion_Status.set_state(next_state)
        
        time = time + DT

        x.append(state.x)
        y.append(state.y)
        yaw.append(state.yaw)
        v.append(state.v)
        t.append(time)
        d.append(di)
        a.append(ai)

        if check_goal(state, goal, index, len(cx)):
            print("Goal")
            break

        if show_animation:  # pragma: no cover
            plt.cla()
            if ox is not None:
                plt.plot(ox, oy, "xr", label="MPC")
            plt.plot(cx, cy, "-r", label="course")
            plt.plot(x, y, "ob", label="trajectory")
            plt.plot(xref[0, :], xref[1, :], "xk", label="xref")
            plt.plot(cx[index], cy[index], "xg", label="target")
            plot_car(state.x, state.y, state.yaw, steer=di)
            '''
            points=gen_points(state.x,state.y)
            ang_yaw=-(state.yaw*180.0)/math.pi
            rp=move_points(points,0,0,ang_yaw)
            r_points=rp.apply()
            points=np.array(points,dtype=np.float64)
            r_points=np.array(r_points,dtype=np.float64)
            plt.plot(r_points[:,0].flatten(),r_points[:,1].flatten(),"-r")
            '''
            plt.axis("equal")
            plt.grid(True)
            plt.title("Time[s]:" + str(round(time, 2))
                      + ", speed[km/h]:" + str(round(state.v * 3.6, 2)))
            plt.pause(0.0001)

if show_animation:  # pragma: no cover
            plt.close("all")
            plt.subplots()
            plt.plot(cx, cy, "-r", label="spline")
            plt.plot(x, y, "-g", label="tracking")
            plt.grid(True)
            plt.axis("equal")
            plt.xlabel("x[m]")
            plt.ylabel("y[m]")
            plt.legend()

            plt.subplots()
            plt.plot(t, v, "-r", label="speed")
            plt.grid(True)
            plt.xlabel("Time [s]")
            plt.ylabel("Speed [kmh]")

            plt.show()


ds 0 0.0
index 0 0
ds 0 0.0
index 0 0
ds 0 0.0016000000000302866
index 0 0
ds 0 0.01440000000032836
index 0 0
ds 0 0.05759999999867224
index 0 0
ds 0 0.1599999999822717
index 0 0
ds 0 0.3599999998830289
index 0 0
ds 1 0.5749083657949627
index 1 1
ds 1 0.2287011784940143
index 1 1
ds 1 0.025035835731058245
index 1 1
ds 1 0.04071233917007465
index 1 1
ds 1 0.36213069019530114
index 1 1
ds 2 0.30518860547625315
index 2 2
ds 2 0.005247808642630841
index 2 2
ds 2 0.20031231020165713
index 2 2
ds 3 0.33480182545013903
index 3 3
ds 3 0.0004604862890073031
index 3 3
ds 3 0.43743157883386324
index 3 3
ds 4 0.0541845667617246
index 4 4
ds 4 0.23742560708231475
index 4 4
ds 5 0.09663570125820202
index 5 5
ds 5 0.2393261195632441
index 5 5
ds 6 0.043671312772284324
index 6 6
ds 6 0.4505617427186385
index 6 6
ds 7 0.006087360600348691
index 7 7
ds 8 0.20139719626007727
index 8 8
ds 8 0.30430225632118185
index 8 8
ds 9 0.01930653151004376
index 9 9
ds 10 0.07164302098242621
index 10 10
ds 11 0.37705

In [11]:
from scipy.interpolate import CubicSpline
ax = [0.0, -10.0, -20.0, -40.0, -50.0, -60.0, -70.0]
ay = [0.0, -1.0, 1.0, 0.0, -1.0, 1.0, 0.0]
cx, cy, cyaw, ck, s = cubic_spline_planner.calc_spline_course(ax, ay, ds=dl)
plt.plot(ax,ay,"-b")
plt.plot(cx,cy,"-r")
csx=CubicSpline(range(len(ax)),ax)
csy=CubicSpline(range(len(ay)),ay)

plt.plot(csx,csy,"-g")
plt.show()

TypeError: float() argument must be a string or a number, not 'CubicSpline'

In [19]:
import paho.mqtt.client as mqtt #import the client1
import time
broker_address="192.168.11.3" 
#broker_address='Lenovo-Yoga-2-11'
#broker_address="test.mosquitto.org" #use external broker
client = mqtt.Client("P1") #create new instance
client.connect(broker_address) #connect to broker
i=10
while(i):
    client.publish(topic="house/bulbs/bulb1",payload="OFF")#publis
    time.sleep(2)
    i-=1

In [7]:
import ssl
import sys

import paho.mqtt.client
import paho.mqtt.publish
import socket
hostname=socket.gethostname()
def on_connect(client, userdata, flags, rc):
	print('connected')

def _main():
	paho.mqtt.publish.single(
		topic='[topic]',
		payload='[message]',
		qos=2,
		hostname=hostname,
		port=8883,
		client_id='[clientid]',
		
	)

if __name__ == '__main__':
	_main()
	sys.exit(0)


ConnectionRefusedError: [Errno 111] Connection refused

In [6]:
print(

SyntaxError: unexpected EOF while parsing (<ipython-input-6-424fbb3a34c5>, line 1)

In [ ]:
%matplotlib tk
from matplotlib import pyplot as plt
import time
ang=0
pos=0
#plt.ion()
cx, cy, cyaw, ck = get_switch_back_course(1.0)

    
for i in range(100):
    plt.cla()
    frame=np.zeros((100,100),dtype=np.uint8)
    points=np.array([(20,20),(20,40),(40,40),(40,20)],dtype=np.float64)
    points=gen_points(cx[i],cy[i])
    #rp=rotate_point(points,ang)
    yaw=(cyaw[i]*180.0)/math.pi
    rp=move_points(points,0,0,yaw)
    ang+=10
    pos+=0
    r_points=rp.apply()
    points=np.array(points,dtype=np.int32)
    r_points=np.array(r_points,dtype=np.int32)
    cv2.polylines(frame,[(points)],True,(255,0,0),5)
    cv2.polylines(frame,[(r_points)],True,(0,255,0),5)
    #cv2.drawContours(img, contours, -1, (0,255,0), 3)
    plt.title(str(yaw))
    plt.plot(r_points[:,0],r_points[:,1])
    plt.plot(cx,cy)
    
    plt.pause(0.0001)
    #plt.show()
    time.sleep(.1)

In [ ]:
cyaw

In [ ]:
class rotate_point():
    def __init__(self,points,phi):
        self.points=points
        self.shifted_points=[]
        self.rev_shifted_points=[]
        self.rot_points=[]
        self.n=len(points)
        self.ox=0
        self.oy=0
        phi=(math.pi*phi)/180
        self.rot_mat=np.array([[math.cos(phi),-math.sin(phi)],[math.sin(phi),math.cos(phi)]])
    def get_origin(self):
        ox=0
        oy=0
        for point in self.points:
            ox+=point[0]
            oy+=point[1]
        self.ox=ox/self.n
        self.oy=oy/self.n
        print(self.ox,self.oy)
        return self.ox,self.oy
    def shift_origin(self):
        for point in self.points:
            self.shifted_points.append((point[0]-self.ox,point[1]-self.oy))
        return self.shifted_points
    def reverse_shift_origin(self):
          for point in self.rot_points:
            pt=(int(point[0]+self.ox),int(point[1]+self.oy))
            self.rev_shifted_points.append(pt)
          return self.rev_shifted_points
    def rotate(self):
        for point in self.shifted_points:
            point=np.array(point)
            self.rot_points.append(point.dot(self.rot_mat))
        return self.rot_points
    def apply(self):
        self.get_origin()
        self.shift_origin()
        self.rotate()
        self.reverse_shift_origin()
        return self.rev_shifted_points

In [ ]:
dd=np.ones((3,2),dtype=np.uint8)
np.sum(dd,axis=0)

In [ ]:
import cv2
import numpy as np
import imutils
import math
img=np.ones((640,480,3),dtype=np.uint8)
car=np.ones((60,40,3),dtype=np.uint8)
car[:,:,0]=255
#img=img+(.1)



class move_points():
    def __init__(self,points,XX,YY,phi):
        self.points=points.copy()
        self.XX=XX
        self.YY=YY
        self.ox=0
        self.oy=0
        phi=(math.pi*phi)/180
        self.rot_mat=np.array([[math.cos(phi),-math.sin(phi)],[math.sin(phi),math.cos(phi)]])
    def move(self):
        self.points[:,0]+=self.XX
        self.points[:,1]+=self.YY
    def get_origin(self):
        ox=0
        oy=0
        oo=np.mean(self.points,axis=0)
        self.ox=oo[0]
        self.oy=oo[1]
        print(self.ox,self.oy)
        return self.ox,self.oy
    def shift_origin(self):
        self.points[:,0]-=np.float64(self.ox)
        self.points[:,1]-=np.float64(self.oy)
        return self.points
    def reverse_shift_origin(self):
        self.points[:,0]+=self.ox
        self.points[:,1]+=self.oy
        return self.points
    def rotate(self):
        for i in range(len(self.points)):
            self.points[i]=self.points[i].dot(self.rot_mat)
        return self.points
    def apply(self):
        self.move()
        self.get_origin()
        self.shift_origin()
        self.rotate()
        self.reverse_shift_origin()
        return self.points
    
def deg2rad(ang):
    return (math.pi*ang)/180
yaw=deg2rad(20)
x1=200
y1=200
x2=300
y2=200

        
ang=10
pos=0
for i in range(30):
    
    #car=imutils.rotate_bound(car,10)
    dx,dy,_=np.shape(car)
    print(dx,dy)
    #img[0:dx,0:dy]=car
    #cv2.rectangle(img,(0,0),(40,40),(255,0,0),1)
    points=np.array([(100,100),(100,200),(200,200),(200,100)],dtype=np.float64)
    #rp=rotate_point(points,ang)
    rp=move_points(points,pos,0,ang)
    ang+=10
    pos+=10
    r_points=rp.apply()
    x1=int(x1)
    y1=int(y1)
    x2=int(x2)
    y2=int(y2)
    #points=np.array(points)
    #points = points.reshape((-1,1,2))

    #r_points=np.array(r_points)
    #r_points = r_points.reshape((-1,1,2))
    points=np.array(points,dtype=np.int32)
    r_points=np.array(r_points,dtype=np.int32)
    
    cv2.polylines(img,[(points)],True,(255,0,0),1)
    cv2.polylines(img,[(r_points)],True,(0,255,0),1)
    cv2.imshow('img',img)
    #cv2.imshow('img',img)
    cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
plist=np.array([(100,100),(100,200)],dtype=np.float64)
#plist=[(100,100),(100,200)]
def ddeg2rad(ang):
    return (math.pi*ang)/180
rp=rotate_point(plist,30)
rp=move_points(plist,0,0,30)
rp.apply()

In [ ]:
def rotate_bound(image, angle):
    # grab the dimensions of the image and then determine the
    # center
    (h, w) = image.shape[:2]
    (cX, cY) = (w // 2, h // 2)
 
    # grab the rotation matrix (applying the negative of the
    # angle to rotate clockwise), then grab the sine and cosine
    # (i.e., the rotation components of the matrix)
    M = cv2.getRotationMatrix2D((cX, cY), -angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])
 
    # compute the new bounding dimensions of the image
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))
 
    # adjust the rotation matrix to take into account translation
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY
 
    # perform the actual rotation and return the image
    return cv2.warpAffine(image, M, (nW, nH))


In [ ]:
np.shape(car)

In [ ]:
len(cx)